This code is supposed to perform an average resource allocation bipartite projection
For reference please read the book. The projection is going to be made **onto hosts**, because we found that doing it on parasites poses problems with a) similarity b) creating an answerable hypothesis.

Why do we use the average resource allocation to discount both for saturation problem and bandwidth problem.
Bandwidth problem: when there is a very rare parasites, which feeds upon only these two animals there is an argument that these animals are more similar that animals sharing a parasite which feeds upon 1000 other species.
Saturation problem: when these animals share 60% and 80% percent of parasites, there is an argument that these animals are more similar than animals sharing only 20% and 10% of animals.

Why do we use average, not minimum or maximum?
- minimum would mean taking the weight of directed link from the animal with lower share, maximum with greater. Minimum means that we can safely assume that both of them are ???? TO do - 
- average - ok, fine.
I think taking the minimum could make sense because it guarantees that both species are similar of at least this bar to each other.
But for now we go with average



You project hosts onto parasites 
then, you want to answer if this projection was good?
Maybe it's OK?




In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from networkx.algorithms import bipartite
import pickle


In [2]:
df = pd.read_csv('./full_data.csv')
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Host', 'Parasite', 'ParasiteFull',
       'group', 'locality', 'hostgroup', 'animal_nodes', 'parasites_nodes'],
      dtype='object')

In [3]:
df['hostgroup'].unique()

array(['Chondrichthyes', 'Teleostei', 'Mammalia', 'Aves', 'Malacostraca',
       'Reptilia', 'Amphibia', 'Maxillopoda', 'Chondrostei',
       'Cephalaspidomorphi', 'Bivalvia', 'Cephalopoda', 'Holostei',
       'Insecta', 'Diplopoda', 'Gastropoda', 'Ostracoda', 'Clitellata',
       'Branchiopoda', 'Cladistei', 'Arachnida', 'Polychaeta', 'Enoplea',
       'Echinoidea', 'Nuda', 'Tentaculata', 'Ophiuroidea', 'Dipnoi',
       'Sagittoidea'], dtype=object)

Below I'm assigning every property to the host, such that after projection we loose no data.
Remember:
- animalis might be victims of multiple paragroups
- animals are assigned to one hostgroup.
- animals might be in multiple localities.
- parasites are assigned to one hostgroup and one paragroup and multiple localities

In [4]:
#projection of parasites onto hosts
#1-Mode projection
G = nx.Graph()

for i, el in df.iterrows():
    if G.has_node(el['Host']):
        G.add_node(el['Host'], bipartite=0)
    if G.has_node(el['Parasite']):
        G.nodes[el['Parasite']]['locality'].add(el['locality'])
        G.nodes[el['Parasite']]['host_group'].add(el['hostgroup'])
    else:
        locality = set()
        locality.add(el['locality'])
        host_group = set()
        host_group.add(el['hostgroup'])

        G.add_node(el['Parasite'], bipartite=1, host_group=host_group, para_group=el['group'], locality=locality)
    if not G.has_edge(el['Parasite'], el['Host']):
        G.add_edge(el['Parasite'], el['Host'], )


bipartite_1_nodes = [n for n, data in G.nodes(data=True) if data.get('bipartite') == 1]

print(len(bipartite_1_nodes), bipartite_1_nodes)

projected_G = bipartite.projected_graph(G, bipartite_1_nodes)

print(len(projected_G.nodes(data=True)) ,len(projected_G.edges(data=True)))

# Copy attributes from the original graph G to projected_G
for node in projected_G.nodes():
    if node in G.nodes():
        # Copy all attributes of the node from the original graph
        projected_G.nodes[node].update(G.nodes[node])

# Save the graph to a file
with open('projected_graph_para.gpickle', 'wb') as f:
    pickle.dump(projected_G, f, protocol=pickle.HIGHEST_PROTOCOL)



14720 ['Acanthocephaloides incrassatus', 'Acanthocephaloides propinquus', 'Acanthogyrus lizae', 'Neoechinorhynchus agilis', 'Telosentis exiguus', 'Acanthogyrus malawiensis', 'Acanthogyrus tilapiae', 'Eocollis arcanus', 'Eocollis catostomi', 'Leptorhynchoides aphredoderi', 'Acanthocephalus rauschi', 'Corynosoma clavatum', 'Corynosoma semerme', 'Corynosoma strumosum', 'Corynosoma validum', 'Corynosoma villosum', 'Corynosoma constrictum', 'Echinorhynchus gadi', 'Echinorhynchus salmonis', 'Mediorhynchus robustus', 'Neoechinorhynchus cristatus', 'Neoechinorhynchus rutili', 'Neoechinorhynchus strigosus', 'Polymorphus marilis', 'Polymorphus paradoxus', 'Pomphorhynchus bulbocolli', 'Southwellina sacra', 'Andracantha baylisi', 'Andracantha clavata', 'Bolbosoma balaenae', 'Bolbosoma brevicolle', 'Bolbosoma hamiltoni', 'Bolbosoma physeteris', 'Bolbosoma tuberculata', 'Bolbosoma turbinella', 'Corynosoma arctocephali', 'Corynosoma australe', 'Corynosoma bullosum', 'Corynosoma evae', 'Corynosoma ham

In [48]:
for i in projected_G.nodes(data=True):
    print(i)

('Acanthocephaloides incrassatus', {'bipartite': 1, 'host_group': {'Teleostei', 'Chondrichthyes'}, 'para_group': 'Acanthocephala', 'locality': {'Azov Sea', 'Black Sea', 'Adriatic Sea', 'Atlantic Ocean'}})
('Acanthocephaloides propinquus', {'bipartite': 1, 'host_group': {'Teleostei', 'Chondrichthyes'}, 'para_group': 'Acanthocephala', 'locality': {'Azov Sea', 'Northeast Atlantic', 'France', 'Western Europe', 'Atlantic Ocean', 'Sea of Marmara', 'Black Sea', 'Spain + Andalusia', 'Western Mediterranean', 'Adriatic Sea'}})
('Acanthogyrus lizae', {'bipartite': 1, 'host_group': {'Teleostei', 'Maxillopoda'}, 'para_group': 'Acanthocephala', 'locality': {'English Channel', 'Northeast Atlantic', 'Adriatic Sea', 'Atlantic Ocean'}})
('Neoechinorhynchus agilis', {'bipartite': 1, 'host_group': {'Teleostei'}, 'para_group': 'Acanthocephala', 'locality': {'Iraq + Kuwait', 'Iraq', 'Pacific Ocean', 'Middle East + Asia Minor', 'Australia', 'Northeast Atlantic', 'Atlantic Ocean', 'Black Sea', 'Sardinia', 'It

In [ ]:
# Projection of parasites onto hosts (hosts remain)
G = nx.Graph()

for i, el in df.iterrows():
    if G.has_node(el['Parasite']):
        G.add_node(el['Parasite'], bipartite=0)
    if G.has_node(el['Host']):
        G.nodes[el['Host']]['locality'].add(el['locality'])
        G.nodes[el['Host']]['para_group'].add(el['group'])
    else:
        locality = set()
        locality.add(el['locality'])
        para_group = set()
        para_group.add(el['group'])
        G.add_node(el['Host'], bipartite=1, host_group=el['hostgroup'], para_group=para_group, locality=locality)
    if not G.has_edge(el['Host'], el['Parasite']):
        G.add_edge(el['Host'], el['Parasite'], )

In [6]:
max(dict(projected_G.degree()).values())

2260

In [7]:
def stats(graph):
    graph_degrees=dict(graph.degree()).values()
    stats_dict={}
    #average degree
    stats_dict["average_degree"]=sum(graph_degrees)/graph.number_of_nodes()
    #max degree
    stats_dict["max_degree"]=max(graph_degrees)
    #min degree
    stats_dict["min_degree"]=min(graph_degrees)
    #percentage under the average
    stats_dict["percentage_under_average"]=len([d for d in graph_degrees if d < stats_dict["average_degree"]])/len(graph_degrees)
    #assortativity
    stats_dict["degree_assortativity"]=nx.degree_assortativity_coefficient(graph)
    #modularity
    communities = nx.community.greedy_modularity_communities(graph)
    stats_dict["modularity"]=nx.community.modularity(graph,communities)
    #...
    return stats_dict


In [8]:
s=stats(projected_G)

{'average_degree': 71.85611413043478,
 'max_degree': 2260,
 'min_degree': 0,
 'percentage_under_average': 0.73125,
 'degree_assortativity': 0.08475397231055315,
 'modularity': 0.5984877392234296}

In [ ]:
import json
with open('stats/1-mode projection.txt', 'w+') as convert_file: 
     convert_file.write(json.dumps(s))

In [4]:
# average resource allocation bipartite projection for host nodes
# ProbS you multiply the stochastic with a stochastic version of the transpose. then get (wu,v + wv,u)/2
# code below:

bipartite_one_nodes = [ n for n, d in G.nodes(data=True) if d.get('bipartite') == 1]
print(len(bipartite_one_nodes))
badj = nx.bipartite.biadjacency_matrix(G, row_order=sorted(bipartite_one_nodes)).toarray()

14720


In [129]:
# row stochastic adjacency matrix, of going from host to parasite
A = badj / badj.sum(axis=1)[:, np.newaxis]
At = badj.T / badj.T.sum(axis=1)[:, np.newaxis]
weights = A @ At

: 

: 

In [10]:
for el in G.nodes(data=True):
    print(el)
    break

('Acanthocephaloides incrassatus', {'bipartite': 1, 'host_group': {'Chondrichthyes', 'Teleostei'}, 'para_group': 'Acanthocephala', 'locality': {'Azov Sea', 'Adriatic Sea', 'Atlantic Ocean', 'Black Sea'}})


In [54]:
# now we have directional weights, we need to get the average
resource_allocation_weights = (weights + weights.T) / 2
# sanity check: symmetric matrix
print(np.allclose(resource_allocation_weights, resource_allocation_weights.T))
# compute the number of edges
print(np.count_nonzero(~np.isclose(resource_allocation_weights,0)) / 2)


True
225108.0


In [47]:
sum(resource_allocation_weights[1, :])

np.float64(0.5832543940993622)

SELF LOOPS, are we care about them? not much tbh, we can remove them later.
We don't care - we need to remove them later.

In [45]:

# are there any nans in resource_allocation_weights?
print(np.isnan(resource_allocation_weights).any())
# are there any inf in resource_allocation_weights?
print(np.isinf(resource_allocation_weights).any())
# are there any rows that sum to zero? (contains only zeros)
print(np.any(np.isclose(resource_allocation_weights.sum(axis=1), 0)))
# are there any rows which have only one non-zero element?
print(np.any(np.count_nonzero(resource_allocation_weights, axis=1) == 1))

# are there rows not summing to 1?
print(np.any(np.isclose(resource_allocation_weights.sum(axis=1), 1) == False))

False
False
False
True
True


In [55]:
# do the noise corrected disparity filter
# You don't need to take care of zeros, because these are going to be zero anyway after the filtering


node_edges_weights_sum = resource_allocation_weights.sum(axis=1)
edge_neighbourhood_weights_sums = node_edges_weights_sum[:, np.newaxis] - resource_allocation_weights
# are there rows that sum to zero? (contains only zeros)
print(np.any(np.isclose(edge_neighbourhood_weights_sums.sum(axis=1), 0)))

edge_neighbourhood_weights_means = edge_neighbourhood_weights_sums / (np.count_nonzero(resource_allocation_weights, axis=1) - 1)[:, np.newaxis]
# there could be 0/0, which would result in nan, replace these with zeros
# I don't understand how infs emerge, but they do, so replace them with zeros
edge_neighbourhood_weights_means = np.nan_to_num(edge_neighbourhood_weights_means, nan=0, posinf=0, neginf=0)

# above should not be symmetric
print(np.allclose(edge_neighbourhood_weights_means, edge_neighbourhood_weights_means.T))
disparity_filter = (edge_neighbourhood_weights_means + edge_neighbourhood_weights_means.T) / 2

# where the values are below the disparity filter, set to zero
filtered_weights = resource_allocation_weights.copy()
# check if both disparity filter and resource_allocation_weights are symmetric
print(np.allclose(disparity_filter, disparity_filter.T))
print(np.allclose(resource_allocation_weights, resource_allocation_weights.T))
filtered_weights[resource_allocation_weights < disparity_filter] = 0
filtered_weights

False


C:\Users\macie\AppData\Local\Temp\ipykernel_20684\780204485.py:10: RuntimeWarning: divide by zero encountered in divide
  edge_neighbourhood_weights_means = edge_neighbourhood_weights_sums / (np.count_nonzero(resource_allocation_weights, axis=1) - 1)[:, np.newaxis]
C:\Users\macie\AppData\Local\Temp\ipykernel_20684\780204485.py:10: RuntimeWarning: invalid value encountered in divide
  edge_neighbourhood_weights_means = edge_neighbourhood_weights_sums / (np.count_nonzero(resource_allocation_weights, axis=1) - 1)[:, np.newaxis]


False
True
True


array([[0.67559975, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.08333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.07692308, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333333,
        0.33333333],
       [0.        , 0.        , 0.        , ..., 0.        , 0.33333333,
        0.33333333]])

In [56]:

# check if the filtered weights are symmetric
print(np.allclose(filtered_weights, filtered_weights.T))
# compute the number of values which are not close to zero
print(np.count_nonzero(~np.isclose(filtered_weights, 0))/2)
filtered_weights.shape

True


Filtered weights is not symmetric, but has inconsistent amount of edges? :()

In [94]:
# create a new undirected graph with the filtered weights as adjacency matrix
G_filtered = nx.from_numpy_array(filtered_weights)
# reassign the node names and attributes
G_filtered = nx.relabel_nodes(G_filtered, dict(enumerate(bipartite_one_nodes)))
# add the attributes
for n, d in G.nodes(data=True):
    if d.get('bipartite') == 1:
        # drop the bipartite attribute
        d.pop('bipartite')
        G_filtered.nodes[n].update(d)